# Algorithm for Data Processing

1. Start
2. Read .csv file.
3. Create separate files for each state.
4. Is forecast year=year observed?
5. If yes, go to step 6. If no, go to step 9.
6. n=number of duplicates in Internal Project ID.
If Internal Project ID is unavailable, find duplicates by some other method.
7. New columns for Percentage Error.
8. Percentage Error=1/n*(Forecast-Actual)/Forecast. END
9. years=forecast year- year observed.
10. growth rate= difference between two projections/number of years in between projections.
11. adjusted forecast=forecast AADT X growth rate X years. Go to step 7.


In [1]:
import pandas as pd
import numpy as np

In [3]:
flatdata=pd.read_csv("FlatData.csv")
flatdata.groupby('State').count() #summary of the data available

,BriefDescription,Length_miles,FunctionalClass,ImprovementType,InternalProjectID,main_ProjectInformation_ProjectID,County,TollType,ForecastYear,TrafficForecast,ForecastYearType,YearForecastProduced,ForecastMethodology,SegmentDescription,main_TrafficForecast_ProjectID,SegmentID,Year_ofObservation,Count
State,,,,,,,,,,,,,,,,,,
12.0,2066,1803,1923,0,1923,2066,2066,2066,2060,2066,2066,2060,2042,1188,2066,2066,2066,2066
27.0,38,38,38,0,38,38,38,38,38,38,38,38,38,32,38,38,38,38
28.0,2176,1565,0,0,2176,2176,1987,2176,2176,2176,0,2176,2176,2176,2176,0,2176,2176
41.0,12411,0,12409,0,10677,12413,12413,12413,12413,12413,12413,12413,12413,12413,12413,0,12413,12413
58.0,0,0,457,0,457,457,457,457,457,457,457,457,457,0,457,0,457,457


In [20]:
cleaned.groupby('State').count()

,BriefDescription,Length_miles,FunctionalClass,ImprovementType,InternalProjectID,main_ProjectInformation_ProjectID,County,TollType,ForecastYear,TrafficForecast,ForecastYearType,YearForecastProduced,ForecastMethodology,SegmentDescription,main_TrafficForecast_ProjectID,SegmentID,Year_ofObservation,Count
State,,,,,,,,,,,,,,,,,,
12.0,840,653,697,0,697,840,840,840,840,840,840,838,832,524,840,840,840,840
27.0,8,8,8,0,8,8,8,8,8,8,8,8,8,7,8,8,8,8
28.0,2100,1549,0,0,2100,2100,1911,2100,2100,2100,0,2100,2100,2100,2100,0,2100,2100
41.0,1854,0,1854,0,1584,1854,1854,1854,1854,1854,1854,1854,1854,1854,1854,0,1854,1854
58.0,0,0,457,0,457,457,457,457,457,457,457,457,457,0,457,0,457,457


In [4]:
florida=flatdata[flatdata['State']==12]
michigan=flatdata[flatdata['State']==27]
minnesota=flatdata[flatdata['State']==28]
wisconsin=flatdata[flatdata['State']==58]
ohio=flatdata[flatdata['State']==41]
ohio.head()

,BriefDescription,Length_miles,FunctionalClass,ImprovementType,State,InternalProjectID,main_ProjectInformation_ProjectID,County,TollType,ForecastYear,TrafficForecast,ForecastYearType,YearForecastProduced,ForecastMethodology,SegmentDescription,main_TrafficForecast_ProjectID,SegmentID,Year_ofObservation,Count
0,IR71/US 30 interchange Modification,NaN,1.0,NaN,41.0,11378,41-2-11378-IR71/US 30 interchange Modification,Richland,1,2000.0,37190,1.0,1994.0,6,", , RIC7119.08",2,NaN,2000,42310
1,IR71/US 30 interchange Modification,NaN,1.0,NaN,41.0,11378,41-2-11378-IR71/US 30 interchange Modification,Richland,1,2020.0,45190,3.0,1994.0,6,", , RIC7119.08",2,NaN,2008,39760
2,IR71/US 30 interchange Modification,NaN,3.0,NaN,41.0,11378,41-3-11378-IR71/US 30 interchange Modification,Richland,1,2000.0,12920,1.0,1994.0,6,", , RIC3017.2",3,NaN,2000,15780
3,IR71/US 30 interchange Modification,NaN,3.0,NaN,41.0,11378,41-3-11378-IR71/US 30 interchange Modification,Richland,1,2020.0,17390,3.0,1994.0,6,", , RIC3017.2",3,NaN,2008,15630
4,IR71/US 30 interchange Modification,NaN,1.0,NaN,41.0,11378,41-4-11378-IR71/US 30 interchange Modification,Richland,1,2000.0,37820,1.0,1994.0,6,", , RIC7118.78",4,NaN,2000,42450


In [5]:
#ohio[ohio['InternalProjectID'].isnull()]
flatdata[flatdata['InternalProjectID'].isnull()].groupby('State').count()
#flatdata[flatdata['Count'].isnull()].groupby('State').count()

,BriefDescription,Length_miles,FunctionalClass,ImprovementType,InternalProjectID,main_ProjectInformation_ProjectID,County,TollType,ForecastYear,TrafficForecast,ForecastYearType,YearForecastProduced,ForecastMethodology,SegmentDescription,main_TrafficForecast_ProjectID,SegmentID,Year_ofObservation,Count
State,,,,,,,,,,,,,,,,,,
12.0,143,0,0,0,0,143,143,143,143,143,143,143,143,143,143,143,143,143
41.0,1734,0,1736,0,0,1736,1736,1736,1736,1736,1736,1736,1736,1736,1736,0,1736,1736


The above describes the number of records in our database that don't have any Internal Project ID. We have to analyse these differently. 

In [6]:
cleaned=ohio[ohio['ForecastYear']==ohio['Year_ofObservation']]
#cleaned.head()


Looking at the Ohio dataset, we see that in the original excel worksheetthere is a column titled "Physical Works COmpleted On". If the Near Opening Year Count happens before this date, that means the count is taken when the project is not yet done. SO we have to exclude this value.

In [30]:
flatdata['DiffYear']=flatdata['ForecastYear']-flatdata['Year_ofObservation']

newflatdata=flatdata[flatdata['DiffYear']>=0] #to remove values whose counts are not applicable.

newflatdata=newflatdata[newflatdata['State']!=41]
newflatdata.groupby('State').count()


,BriefDescription,Length_miles,FunctionalClass,ImprovementType,InternalProjectID,main_ProjectInformation_ProjectID,County,TollType,ForecastYear,TrafficForecast,ForecastYearType,YearForecastProduced,ForecastMethodology,SegmentDescription,main_TrafficForecast_ProjectID,SegmentID,Year_ofObservation,Count,DiffYear
State,,,,,,,,,,,,,,,,,,,
12.0,1948,1687,1805,0,1805,1948,1948,1948,1948,1948,1948,1942,1928,1098,1948,1948,1948,1948,1948
27.0,33,33,33,0,33,33,33,33,33,33,33,33,33,27,33,33,33,33,33
28.0,2134,1553,0,0,2134,2134,1945,2134,2134,2134,0,2134,2134,2134,2134,0,2134,2134,2134
58.0,0,0,457,0,457,457,457,457,457,457,457,457,457,0,457,0,457,457,457


Now, newflatdata dataframe contains the projects that have actual counts on or after the forecast year. Note it doesn't include Ohio, since that have to be analysed separately.

#### Here, InternalProjectID are unique identifiers. 

## Separate the newflatdata 

The first condition to be fulfilled is whether the DiffYear is zero. If it is zero, then we don't need to adjust the forecast.

If the DiffYear is not zero, adjust by the aforementioned formula.

In [42]:
estimation=newflatdata[newflatdata['DiffYear']==0]
estimation.groupby('State').count()
estimation.head()

,BriefDescription,Length_miles,FunctionalClass,ImprovementType,State,InternalProjectID,main_ProjectInformation_ProjectID,County,TollType,ForecastYear,TrafficForecast,ForecastYearType,YearForecastProduced,ForecastMethodology,SegmentDescription,main_TrafficForecast_ProjectID,SegmentID,Year_ofObservation,Count,DiffYear
12413,NaN,NaN,3.0,NaN,58.0,1663,58-6231-1663-,Washington,1,2010.0,5300,1.0,2007.0,1,NaN,6231,NaN,2010,3700,0.0
12414,NaN,NaN,3.0,NaN,58.0,1663,58-6232-1663-,Washington,1,2010.0,7400,1.0,2007.0,1,NaN,6232,NaN,2010,5500,0.0
12415,NaN,NaN,3.0,NaN,58.0,1663,58-6233-1663-,Washington,1,2010.0,4150,1.0,2007.0,1,NaN,6233,NaN,2010,2800,0.0
12416,NaN,NaN,3.0,NaN,58.0,1663,58-6234-1663-,Washington,1,2010.0,3700,1.0,2007.0,1,NaN,6234,NaN,2010,2500,0.0
12417,NaN,NaN,3.0,NaN,58.0,1663,58-6235-1663-,Washington,1,2010.0,4500,1.0,2007.0,1,NaN,6235,NaN,2010,3500,0.0


In [32]:
estimation2=newflatdata[newflatdata['DiffYear']>0]
estimation2.groupby('State').count()

,BriefDescription,Length_miles,FunctionalClass,ImprovementType,InternalProjectID,main_ProjectInformation_ProjectID,County,TollType,ForecastYear,TrafficForecast,ForecastYearType,YearForecastProduced,ForecastMethodology,SegmentDescription,main_TrafficForecast_ProjectID,SegmentID,Year_ofObservation,Count,DiffYear
State,,,,,,,,,,,,,,,,,,,
12.0,1108,1034,1108,0,1108,1108,1108,1108,1108,1108,1108,1104,1096,574,1108,1108,1108,1108,1108
27.0,25,25,25,0,25,25,25,25,25,25,25,25,25,20,25,25,25,25,25
28.0,34,4,0,0,34,34,34,34,34,34,0,34,34,34,34,0,34,34,34


In [41]:
estimation2=estimation2[estimation2['ForecastYearType']!=3] #taking the opening year forecast only.
estimation2.head()

,BriefDescription,Length_miles,FunctionalClass,ImprovementType,State,InternalProjectID,main_ProjectInformation_ProjectID,County,TollType,ForecastYear,TrafficForecast,ForecastYearType,YearForecastProduced,ForecastMethodology,SegmentDescription,main_TrafficForecast_ProjectID,SegmentID,Year_ofObservation,Count,DiffYear
13416,TH 35 and Little Canada Road,NaN,NaN,NaN,28.0,TA-M286,28-7234-TA-M286-TH 35 and Little Canada Road,Ramsey,1,2005.0,3600,NaN,1981.0,1,", , , Little Canada RdWest of Co Rd CEB",7234,NaN,2004,20000,1.0
13417,TH 35 and Little Canada Road,NaN,NaN,NaN,28.0,TA-M286,28-7235-TA-M286-TH 35 and Little Canada Road,Ramsey,1,2005.0,3600,NaN,1981.0,1,", , , WB",7235,NaN,2004,20000,1.0
13418,TH 35 and Little Canada Road,NaN,NaN,NaN,28.0,TA-M286,28-7236-TA-M286-TH 35 and Little Canada Road,Ramsey,1,2005.0,6000,NaN,1981.0,1,", , , Little Canada RdBetween Co Rd C & Lakesh...",7236,NaN,2004,20000,1.0
13419,TH 35 and Little Canada Road,NaN,NaN,NaN,28.0,TA-M286,28-7237-TA-M286-TH 35 and Little Canada Road,Ramsey,1,2005.0,6000,NaN,1981.0,1,", , , WB",7237,NaN,2004,20000,1.0
13856,TH 494 from TH 12 (394) to TH 55,NaN,NaN,NaN,28.0,TA-M358,28-7674-TA-M358-TH 494 from TH 12 (394) to TH 55,Hennepin,1,2010.0,32800,NaN,1987.0,1,", , , I-494Between I-394NB",7674,NaN,1990,27000,20.0


In [40]:
project_weight = estimation['InternalProjectID'].groupby(estimation['InternalProjectID']).count().to_frame(name='n')
project_weight

,n
InternalProjectID,
1663,5
1693,5
1745,12
1760,5
1864,1
1905,2
1929,23
1939,4
1959,8


In [43]:
newestimation1 = pd.merge(estimation,project_weight, left_on='InternalProjectID', right_index=True)
newestimation1

,BriefDescription,Length_miles,FunctionalClass,ImprovementType,State,InternalProjectID,main_ProjectInformation_ProjectID,County,TollType,ForecastYear,...,ForecastYearType,YearForecastProduced,ForecastMethodology,SegmentDescription,main_TrafficForecast_ProjectID,SegmentID,Year_ofObservation,Count,DiffYear,n
12413,NaN,NaN,3.0,NaN,58.0,1663,58-6231-1663-,Washington,1,2010.0,...,1.0,2007.0,1,NaN,6231,NaN,2010,3700,0.0,5
12414,NaN,NaN,3.0,NaN,58.0,1663,58-6232-1663-,Washington,1,2010.0,...,1.0,2007.0,1,NaN,6232,NaN,2010,5500,0.0,5
12415,NaN,NaN,3.0,NaN,58.0,1663,58-6233-1663-,Washington,1,2010.0,...,1.0,2007.0,1,NaN,6233,NaN,2010,2800,0.0,5
12416,NaN,NaN,3.0,NaN,58.0,1663,58-6234-1663-,Washington,1,2010.0,...,1.0,2007.0,1,NaN,6234,NaN,2010,2500,0.0,5
12417,NaN,NaN,3.0,NaN,58.0,1663,58-6235-1663-,Washington,1,2010.0,...,1.0,2007.0,1,NaN,6235,NaN,2010,3500,0.0,5
12418,NaN,NaN,4.0,NaN,58.0,2015,58-6236-2015-,Washington,1,2007.0,...,1.0,2005.0,1,NaN,6236,NaN,2007,14000,0.0,3
12419,NaN,NaN,4.0,NaN,58.0,2015,58-6237-2015-,Washington,1,2007.0,...,1.0,2005.0,1,NaN,6237,NaN,2007,14700,0.0,3
12420,NaN,NaN,4.0,NaN,58.0,2015,58-6238-2015-,Washington,1,2007.0,...,1.0,2005.0,1,NaN,6238,NaN,2007,16900,0.0,3
12421,NaN,NaN,0.0,NaN,58.0,2126,58-6239-2126-,Washington,1,2007.0,...,1.0,2006.0,1,NaN,6239,NaN,2007,1900,0.0,10
12423,NaN,NaN,3.0,NaN,58.0,2126,58-6241-2126-,Washington,1,2007.0,...,1.0,2006.0,1,NaN,6241,NaN,2007,5100,0.0,10
